<a href="https://colab.research.google.com/github/26medias/TF-Face-Angle-Translation/blob/master/Face_Dataset_Assembly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Dataset Assembly

In [0]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!rm -r Faces
!wget https://storage.googleapis.com/tf-face-angle-translation/datasets/faces-clustered-720p-large.tar.gz
!wget https://storage.googleapis.com/tf-face-angle-translation/datasets/faces-clustered-720p-large2.tar.gz
!tar -xzf faces-clustered-720p-large.tar.gz
!tar -xzf faces-clustered-720p-large2.tar.gz
!rm faces-clustered-720p-large.tar.gz
!rm faces-clustered-720p-large2.tar.gz

rm: cannot remove 'Faces': No such file or directory
--2019-09-06 02:07:00--  https://storage.googleapis.com/tf-face-angle-translation/datasets/faces-clustered-720p-large.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 2a00:1450:400c:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104053108 (99M) [application/x-tar]
Saving to: ‘faces-clustered-720p-large.tar.gz’

faces-clustered-720 100%[===================>]  99.23M   124MB/s    in 0.8s    

2019-09-06 02:07:01 (124 MB/s) - ‘faces-clustered-720p-large.tar.gz’ saved [104053108/104053108]

--2019-09-06 02:07:03--  https://storage.googleapis.com/tf-face-angle-translation/datasets/faces-clustered-720p-large2.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.140.128, 2a00:1450:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.140.128|:443.

## Assemble the CSV without moving the files

In [3]:
!rm -r Dataset
import csv
import glob
import os


def assemble(outputDirectory="Dataset"):
  if not os.path.isdir(outputDirectory):
    os.mkdir(outputDirectory, 755);
  # Find the CSV files
  CSVs = glob.glob('Faces/*/*.csv')
  # Open a new CSV
  with open(outputDirectory+"/face-dataset.csv", "w") as csvfile:
    fieldnames = ["video_name", "face_group", "image_filename", "embeddings_filename", "landmarks_filename", "segmentations_filename"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for file in CSVs:
      with open(file, newline='') as file:
        reader = csv.reader(file)
        for n, row in enumerate(reader):
          if n> 0:
            writer.writerow({
              "video_name": row[0],
              "face_group": row[1],
              "image_filename": row[2],
              "embeddings_filename": row[3],
              "landmarks_filename": row[4],
              "segmentations_filename": row[5]
            })
    # Check how many rows we have now
  with open(outputDirectory+"/face-dataset.csv", newline='') as outputFile:
    output_reader = csv.reader(outputFile)
    x = 0
    for n, row in enumerate(output_reader):
      x+=1
    print("Output", x-1)

assemble()

rm: cannot remove 'Dataset': No such file or directory
Output 23362


## Assemble & move the files

In [4]:
#

!rm -r FaceDataset

import csv
import glob
import os

def moveRow(row, n, outputDirectory):
  # Rows to move
  moveList = [2,3,4,5]
  filenames = []
  for i in moveList:
    filename = row[i]
    ext = filename.split(".")[-1]
    outputFilename = '{:06d}'.format(n)+'.'+ext
    if i==2:
      directory = 'Faces'
    elif i==3:
      directory = 'Embeddings'
    elif i==4:
      directory = 'Landmarks'
    elif i==5:
      directory = 'Segmentations'
    filename = outputDirectory+"/"+directory+"/"+outputFilename
    if os.path.exists(row[i]):
      os.rename(row[i], filename)
      filenames.append(filename)
  return filenames;

def assemble(outputDirectory="FaceDataset"):
  
  dirFaces         = outputDirectory+"/Faces/"
  dirEmbeddings    = outputDirectory+"/Embeddings/"
  dirLandmarks     = outputDirectory+"/Landmarks/"
  dirSegmentations = outputDirectory+"/Segmentations/"
  
  # Create the directories
  if not os.path.isdir(outputDirectory):
    os.mkdir(outputDirectory, 755);
  if not os.path.isdir(dirFaces):
    os.mkdir(dirFaces, 755);
  if not os.path.isdir(dirEmbeddings):
    os.mkdir(dirEmbeddings, 755);
  if not os.path.isdir(dirLandmarks):
    os.mkdir(dirLandmarks, 755);
  if not os.path.isdir(dirSegmentations):
    os.mkdir(dirSegmentations, 755);
  
  # Find the CSV files
  CSVs = glob.glob('Faces/*/*.csv')
  # Open a new CSV
  with open(outputDirectory+"/face-dataset.csv", "w") as csvfile:
    fieldnames = ["video_name", "face_group", "image_filename", "embeddings_filename", "landmarks_filename", "segmentations_filename"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    # For each csv file...
    n = 0
    for file in CSVs:
      # Open the file
      with open(file, newline='') as file:
        reader = csv.reader(file)
        # For each row
        for x, row in enumerate(reader):
          if x > 0: # Skip the header
            # Move the files in the row
            filenames = moveRow(row, n, outputDirectory)
            if len(filenames) == 4:
              n+=1
              writer.writerow({
                "video_name": row[0],
                "face_group": row[1],
                "image_filename": filenames[0],
                "embeddings_filename": filenames[1],
                "landmarks_filename": filenames[2],
                "segmentations_filename": filenames[3]
              })
    # Check how many rows we have now
  with open(outputDirectory+"/face-dataset.csv", newline='') as outputFile:
    output_reader = csv.reader(outputFile)
    x = 0
    for n, row in enumerate(output_reader):
      x+=1
    print("Output", x-1)

assemble()

rm: cannot remove 'FaceDataset': No such file or directory
Output 23362


In [5]:
import pandas as pd
pd.read_csv("/content/FaceDataset/face-dataset.csv") 

,video_name,face_group,image_filename,embeddings_filename,landmarks_filename,segmentations_filename
0,jojo-rabbit-trailer-2_h720p,0,FaceDataset/Faces/000000.jpg,FaceDataset/Embeddings/000000.npy,FaceDataset/Landmarks/000000.npy,FaceDataset/Segmentations/000000.npy
1,jojo-rabbit-trailer-2_h720p,0,FaceDataset/Faces/000001.jpg,FaceDataset/Embeddings/000001.npy,FaceDataset/Landmarks/000001.npy,FaceDataset/Segmentations/000001.npy
2,jojo-rabbit-trailer-2_h720p,0,FaceDataset/Faces/000002.jpg,FaceDataset/Embeddings/000002.npy,FaceDataset/Landmarks/000002.npy,FaceDataset/Segmentations/000002.npy
3,jojo-rabbit-trailer-2_h720p,0,FaceDataset/Faces/000003.jpg,FaceDataset/Embeddings/000003.npy,FaceDataset/Landmarks/000003.npy,FaceDataset/Segmentations/000003.npy
4,jojo-rabbit-trailer-2_h720p,0,FaceDataset/Faces/000004.jpg,FaceDataset/Embeddings/000004.npy,FaceDataset/Landmarks/000004.npy,FaceDataset/Segmentations/000004.npy
5,jojo-rabbit-trailer-2_h720p,0,FaceDataset/Faces/000005.jpg,FaceDataset/Embeddings/000005.npy,FaceDataset/Landmarks/000005.npy,FaceDataset/Segmentations/000005.npy
6,jojo-rabbit-trailer-2_h720p,0,FaceDataset/Faces/000006.jpg,FaceDataset/Embeddings/000006.npy,FaceDataset/Landmarks/000006.npy,FaceDataset/Segmentations/000006.npy
7,jojo-rabbit-trailer-2_h720p,0,FaceDataset/Faces/000007.jpg,FaceDataset/Embeddings/000007.npy,FaceDataset/Landmarks/000007.npy,FaceDataset/Segmentations/000007.npy
8,jojo-rabbit-trailer-2_h720p,0,FaceDataset/Faces/000008.jpg,FaceDataset/Embeddings/000008.npy,FaceDataset/Landmarks/000008.npy,FaceDataset/Segmentations/000008.npy
9,jojo-rabbit-trailer-2_h720p,0,FaceDataset/Faces/000009.jpg,FaceDataset/Embeddings/000009.npy,FaceDataset/Landmarks/000009.npy,FaceDataset/Segmentations/000009.npy


In [6]:
# Save the faces
!tar -zcf FaceDataset.tar.gz FaceDataset

# Upload to Cloud Storage
!gcloud config set project deep-learning-files
!gsutil cp  ./FaceDataset.tar.gz gs://tf-face-angle-translation/datasets/FaceDataset-720p-small.tar.gz

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud alpha survey

Copying file://./FaceDataset.tar.gz [Content-Type=application/x-tar]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\
Operation completed over 1 objects/354.5 MiB.                                    
